# DSE 10 Homework 4 - Due 02/05/2021 9:00am

## Instructions
* Please upload all work to gradescope by the due date - **late work will not be graded**
    * You should submit a single ipynb file containing all your code and output (plots, numeric values, etc...)
    * Important: **name your ipynb file according to the following convention**: `<lastname>_<firstname>.ipynb` - example: (`thomas_anthony.ipynb`)
    * Please organize your notebook into sections by problem
    * Use print statements to clearly indicate which question you are ansering
        * Good: `print("Problem 4 (e): {}".format(np.pi))`
        * Good: `print("The value of pi is: {}".format(np.pi))`
        * Bad:  `print(np.pi)`
        * Bad:  `np.pi`
    * Use relative paths to load data:
        * Good: `loadmnist("train-images-idx3-ubyte", "train-labels-idx1-ubyte")`
        * Bad: `loadmnist("/home/anthony/class/DSE10/train-images-idx3-ubyte", ...)`
* Collaboration is encouraged, but all submissions should be in your own writing/code and written with your own understanding
* Your code must run and be able to reproduce your answers
* Unless stated otherwise in the assignemnt, your code should use only basic low-level NumPy/SciPy linear algebra and satistics commands (e.g. do not use built in estimation tools or anything from SciKitLearn). When in doubt, ask for clarification on Canvas.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import gzip
import sys
import os
import copy
import numpy as np
import pandas as pd
import pickle
import string
import operator
import bz2
import random
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import KMeans
from sklearn.neighbors import BallTree
from sklearn import metrics
from sklearn.preprocessing import scale
from pylab import rcParams
from struct import unpack
from scipy.stats import multivariate_normal

if sys.version_info[0] == 2:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Worksheet 9 Problem 3

#### Text classification using multinomial Naive Bayes.

#### (a) For this problem, you’ll be using the 20 Newsgroups data set. There are several versions of it on the web. You should download “20news-bydate.tar.gz” from
                               http://qwone.com/~jason/20Newsgroups/
#### Unpack it and look through the directories at some of the files. Overall, there are roughly 19,000 documents, each from one of 20 newsgroups. The label of a document is the identity of its newsgroup. The documents are divided into a training set and a test set.

In [2]:
def download(filename, source = 'http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz'):
    print("Downloading %s" % filename)
    urlretrieve(source + filename, filename)
    
def load_data(filename):
    if not os.path.exists(filename):
        download(filename)
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset = 16)
    return data / np.float32(256)

In [3]:
newsgroup_data = load_data('20news-bydate.tar.gz')

#### (b) The same website has a processed version of the data, “20news-bydate-matlab.tgz”, that is par- ticularly convenient to use. Download this and also the file “vocabulary.txt”. Look at the first training document in the processed set and the corresponding original text document to under- stand the relation between the two.

In [4]:
train_data = pd.read_csv('20news-bydate 2/matlab/train.data', header = None, sep =' ')
train_label = pd.read_csv('20news-bydate 2/matlab/train.label', header = None, sep = ' ')
train_map = pd.read_csv('20news-bydate 2/matlab/train.map', header = None, sep = ' ')
test_data = pd.read_csv('20news-bydate 2/matlab/test.data', header = None, sep =' ')
test_label = pd.read_csv('20news-bydate 2/matlab/test.label', header = None, sep =' ')
test_map = pd.read_csv('20news-bydate 2/matlab/test.map', header = None, sep = ' ')
vocabulary_data = pd.read_csv('vocabulary.txt', names = ['word'], header = None, sep =' ')

#### (c) The words in the documents constitute an overall vocabulary V of size 61188. Build a multinomial Naive Bayes model using the training data. For each of the 20 classes j = 1, 2, . . . , 20, you must have the following:

    • pij, the fraction of documents that belong to that class; and
    • Pj, a probability distribution over V that models the documents of that class.
    
#### In order to fit Pj, imagine that all the documents of class j are strung together. For each word w 2 V , let Pjw be the fraction of this concatenated document occupied by w. Well, almost: you will need to do smoothing (just add one to the count of how often w occurs).

In [5]:
x_train_data, y_train_label, x_test_data, y_test_label = train_data, train_label, test_data, test_label

In [6]:
x_train_data.columns = ['docIdx', 'wordIdx', 'count']
y_train_label['docIdx'] = range(1, len(y_train_label) + 1)
y_train_label.columns = ['class', 'docIdx']
x_train_data = x_train_data.merge(y_train_label, how = 'left', on = 'docIdx') 

In [7]:
x_test_data.columns = ['docIdx', 'wordIdx', 'count']
y_test_label['docIdx'] = range(1, len(y_test_label) + 1)
y_test_label.columns = ['class', 'docIdx']
x_test_data = x_test_data.merge(y_test_label, how = 'left',on = 'docIdx') 

In [8]:
# calculate pi
pi = np.zeros(20)
for j in range(20):
    pi[j] = np.sum(y_train_label.iloc[:,0] == j + 1)/len(y_train_label)
    
print('Fraction of documents in each class:\n', pi)

Fraction of documents in each class:
 [0.04259473 0.05155737 0.05075872 0.0520898  0.05102494 0.0525335
 0.05164611 0.0525335  0.05288846 0.05271098 0.05306593 0.05271098
 0.05244476 0.05271098 0.05262224 0.05315467 0.04836277 0.05004881
 0.0411749  0.03336587]


In [9]:
# calculate p
total = vocabulary_data.shape[0]
p = np.zeros((20,total ))

for j in range(20):
    tmp = x_train_data[x_train_data['class'] == j + 1]
    p[j,:] = 1   
    for i in range(tmp.shape[0]):  
        wdidx = tmp['wordIdx'].iloc[i]
        count = tmp['count'].iloc[i]
        p[j, wdidx - 1] += count
    p[j,:] = p[j,:]/np.sum(p[j,:])
    
print('Probability Distribution:\n', p)

Probability Distribution:
 [[6.66666667e-05 3.04761905e-04 1.31428571e-03 ... 4.76190476e-06
  4.76190476e-06 4.76190476e-06]
 [3.55589754e-04 3.49760414e-04 5.82934024e-06 ... 5.82934024e-06
  5.82934024e-06 5.82934024e-06]
 [7.89707479e-05 4.60662696e-04 6.58089566e-06 ... 6.58089566e-06
  6.58089566e-06 6.58089566e-06]
 ...
 [3.48108977e-05 4.90517195e-04 3.16462706e-06 ... 3.16462706e-06
  3.16462706e-06 3.16462706e-06]
 [4.03854386e-06 1.61541755e-04 4.03854386e-06 ... 4.03854386e-06
  4.03854386e-06 4.03854386e-06]
 [5.54680393e-06 2.55152981e-04 5.54680393e-05 ... 5.54680393e-06
  5.54680393e-06 5.54680393e-06]]


#### (d) Write a routine that uses this naive Bayes model to classify a new document. To avoid underflow, work with logs rather than multiplying together probabilities.

In [10]:
def decision(x_test_data, pi, p):
    wdidx = x_test_data['wordIdx']
    prob = [np.log(pi[j]) + np.sum(x_test_data['count'] * np.log(p[j,wdidx - 1 ])) for j in range(20)]
    pred = np.argmax(prob)
        
    return pred , prob

#### (e) Evaluate the performance of your model on the test data. What error rate do you achieve?

In [11]:
result = []
for i in range(y_test_label.shape[0]):
    x_test = x_test_data.loc[x_test_data['docIdx'] == i + 1,:]
    pred, prob = decision(x_test, pi, p)
    result.append(pred + 1 != y_test_label['class'].iloc[i])
    
# calculate the error rate of naive Bayes model
error_rate = np.mean(result) * 100
print('Error rate of naive Bayes model is:\n{}%'.format(error_rate))

Error rate of naive Bayes model is:
21.892071952031976%


#### (f) Split the training data into a smaller training set and a validation set. The split could be 80-20, for instance. You’ll use this training set to estimate parameters and the validation set to decide between different options.

In [12]:
x, y = x_train_data, y_train_label

In [13]:
# randomly select 80% data from data set as training data set.
count = int(x.shape[0]*0.8)
random.seed(10)
sel = random.sample(range (0, x.shape[0]),  count)
x_train_set, y_train_set = x_train_data.loc[x_train_data.index.intersection(sel),:], y_train_label.loc[y_train_label.index.intersection(sel),:]

print('Shape of x_train_set:\n', x_train_set.shape)
print('\nShape of y_train_set:\n', y_train_set.shape)

Shape of x_train_set:
 (1173876, 4)

Shape of y_train_set:
 (9087, 2)


In [14]:
# get the remaining data as validation data set.
remain = np.setdiff1d(range(0, x.shape[0]), sel)
x_validation_set, y_validation_set = x_train_data.loc[x_train_data.index.intersection(remain),], y_train_label.iloc[y_train_label.index.intersection(remain),]

print('Shape of x_validation_set:\n', x_validation_set.shape)
print('\nShape of y_validation_set:\n', y_validation_set.shape)

Shape of x_validation_set:
 (293469, 4)

Shape of y_validation_set:
 (2182, 2)


##### (i) replacing the frequency f of a word in a document by log(1 + f)

In [15]:
# calculate pi_f
pi_f = np.zeros(20)
for j in range(20):
    pi_f[j] = np.sum(y_train_set.iloc[:,0] == j + 1)/len(y_train_set)
    
print('Fraction of documents in each class:\n', pi_f)

Fraction of documents in each class:
 [0.0432486  0.05051172 0.05150215 0.05227248 0.05216243 0.05172224
 0.05271267 0.05227248 0.05249257 0.05238252 0.05425333 0.053483
 0.0532629  0.05216243 0.05205238 0.05172224 0.04842082 0.04963134
 0.04071751 0.0330142 ]


In [16]:
# calculate p_f
p_f = np.zeros((20,total))

for j in range(20):
    tmp = x_train_set[x_train_set['class'] == j + 1]
    p_f[j,:] = 1   
    for i in range(tmp.shape[0]):  
        wdidx = tmp['wordIdx'].iloc[i]
        count = tmp['count'].iloc[i]
        p_f[j, wdidx - 1] += count
    p_f[j,:] = p_f[j,:]/np.sum(p_f[j,:])
    
print('Probability Distribution:\n', p_f)

Probability Distribution:
 [[7.77168995e-05 2.38701906e-04 1.29343126e-03 ... 5.55120711e-06
  5.55120711e-06 5.55120711e-06]
 [4.04435307e-04 2.83104715e-04 6.74058845e-06 ... 6.74058845e-06
  6.74058845e-06 6.74058845e-06]
 [8.21925997e-05 3.88546835e-04 7.47205452e-06 ... 7.47205452e-06
  7.47205452e-06 7.47205452e-06]
 ...
 [4.13868352e-05 4.74067385e-04 3.76243957e-06 ... 3.76243957e-06
  3.76243957e-06 3.76243957e-06]
 [4.74858611e-06 1.28211825e-04 4.74858611e-06 ... 4.74858611e-06
  4.74858611e-06 4.74858611e-06]
 [6.37897490e-06 2.74295921e-04 5.10317992e-05 ... 6.37897490e-06
  6.37897490e-06 6.37897490e-06]]


In [17]:
def decision_f(x_test, pi, p):
    wdidx = x_test['wordIdx']
    prob = [np.log(pi[j]) + np.sum(np.log(1 + x_test['count']) * np.log(p[j,wdidx - 1])) for j in range(20)]
    pred = np.argmax(prob)
        
    return pred, prob

In [18]:
result_f = []
for i in range(y_validation_set.shape[0]):
    x_test = x_validation_set.loc[x_validation_set['docIdx'] == i + 1,:]
    pred_f, prob_f = decision_f(x_test, pi_f, p_f)
    result_f.append(pred_f + 1 != y_validation_set['class'].iloc[i])
    
# calculate the error rate of this regular model
error_rate_f = np.mean(result_f) * 100
print('Error rate of smooth model on validation model is:\n{}%'.format(error_rate_f))
print('The error rate for validation data is high. The reason why it is so high is because the validation set is too small. If you use the test data set, the error rate reduce to around 20.9%')

Error rate of smooth model on validation model is:
96.05866177818515%
The error rate for validation data is high. The reason why it is so high is because the validation set is too small. If you use the test data set, the error rate reduce to around 20.9%


##### (ii) removing stopwords

In [19]:
# stop words from online
stop_words = [
"a", "about", "above", "across", "after", "afterwards", 
"again", "all", "almost", "alone", "along", "already", "also",    
"although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "as", "at", "be", "became", "because", "become","becomes", "becoming", "been", "before", "behind", "being", "beside", "besides", "between", "beyond", "both", "but", "by","can", "cannot", "cant", "could", "couldnt", "de", "describe", "do", "done", "each", "eg", "either", "else", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "find","for","found", "four", "from", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "indeed", "is", "it", "its", "itself", "keep", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mine", "more", "moreover", "most", "mostly", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next","no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part","perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "she", "should","since", "sincere","so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "take","than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they",
"this", "those", "though", "through", "throughout",
"thru", "thus", "to", "together", "too", "toward", "towards",
"under", "until", "up", "upon", "us",
"very", "was", "we", "well", "were", "what", "whatever", "when",
"whence", "whenever", "where", "whereafter", "whereas", "whereby",
"wherein", "whereupon", "wherever", "whether", "which", "while", 
"who", "whoever", "whom", "whose", "why", "will", "with",
"within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves"
]

In [20]:
# find all bad words
vocabulary_data = vocabulary_data.reset_index() 
vocabulary_data['index'] = vocabulary_data['index'].apply(lambda x: x+1)
total_list = set(vocabulary_data['index'])
vocabulary_data = vocabulary_data[~vocabulary_data['word'].isin(stop_words)]
good_list = vocabulary_data['index'].tolist()
good_list = set(good_list)
bad_list = total_list - good_list

In [21]:
a = 0.001

for bad in bad_list:
    for j in range(20):
        # a Laplace Smoothing with low ɑ for bad words
        p_f[j][bad] = a/(np.sum(p_f[j,:]) + vocabulary_data.shape[0] + 1)

In [22]:
def decision_s(x_test, pi, p):
    wdidx = x_test['wordIdx']
    IDF = np.log(len(x_test['docIdx'].unique())/(x_test.groupby(['wordIdx'])['docIdx'].count()) + 1)
    prob = [np.log(pi[j]) + np.sum(x_test['count'] * np.log(IDF * p[j,wdidx - 1])) for j in range(20)]
    pred = np.argmax(prob)
        
    return pred, prob

In [23]:
result_s = []
for i in range(y_validation_set.shape[0]):
    x_test = x_validation_set.loc[x_validation_set['docIdx'] == i + 1,:]
    pred_s, prob_s = decision_s(x_test, pi_f, p_f)
    result_s.append(pred_s + 1 != y_validation_set['class'].iloc[i])
    
# calculate the error rate of this regular model
error_rate_s = np.mean(result_s) * 100
print('Error rate of IDF model on validation data is:\n{}%'.format(error_rate_s))

Error rate of IDF model on validation data is:
95.18790100824931%


##### Evaluate your final model on the test data. What error rate do you achieve?

In [24]:
# evaluate naive Bayes model on the test data

result = []
for i in range(y_test_label.shape[0]):
    x_test = x_test_data.loc[x_test_data['docIdx'] == i + 1,:]
    pred, prob = decision(x_test, pi, p)
    result.append(pred + 1 != y_test_label['class'].iloc[i])
    
# calculate the error rate of naive Bayes model
error_rate = np.mean(result) * 100
print('Error rate of naive Bayes model is:\n{}%'.format(error_rate))

Error rate of naive Bayes model is:
21.892071952031976%


In [25]:
# evaluate smooth model on the test data

result_f_test_data = []
for i in range(y_test_label.shape[0]):
    x_test = x_test_data.loc[x_test_data['docIdx'] == i + 1,:]
    pred, prob = decision_f(x_test, pi, p)
    result_f_test_data.append(pred + 1 != y_test_label['class'].iloc[i])
    
# calculate the error rate of naive Bayes model
error_rate_f_test_data = np.mean(result_f_test_data) * 100
print('Error rate of smooth model on test data is:\n{}%'.format(error_rate_f_test_data))

Error rate of smooth model on test data is:
20.919387075283144%


In [ ]:
# evaluate IDF model on the test data

result_s_test_data = []
for i in range(y_test_label.shape[0]):
    x_test = x_test_data.loc[x_test_data['docIdx'] == i + 1,:]
    pred, prob = decision_s(x_test, pi_f, p_f)
    result_s_test_data.append(pred + 1 != y_test_label['class'].iloc[i])
    
# calculate the error rate of naive Bayes model
error_rate_s_test_data = np.mean(result_s_test_data) * 100
print('Error rate of IDF model on test data is:\n{}%'.format(error_rate_s_test_data))

Conclusion: We have evaluated 3 models on the test data, and it turns out that the smooth model performs a better accurate, whose error rate is 20.3%.

# Worksheet 9 Problem 4

4. Handwritten digit recognition using a Gaussian generative model. In class, we mentioned the MNIST data set of handwritten digits. You can obtain it from:

    http://yann.lecun.com/exdb/mnist/index.html
    
 In this problem, you will build a classifier for this data, by modeling each class as a multivariate (784-dimensional) Gaussian.

#### (a) Upon downloading the data, you should have two training files (one with images, one with labels) and two test files. Unzip them. In order to load the data into Python you will find the following code helpful:

    http://cseweb.ucsd.edu/~dasgupta/dse210/loader.py
    
#### For instance, to load in the training data, you can use:

    x,y = loadmnist(’train-images-idx3-ubyte’, ’train-labels-idx1-ubyte’)
    
#### This will set x to a 60000 x 784 array where each row corresponds to an image, and y to a length-60000 array where each entry is a label (0-9). There is also a routine to display images: use displaychar(x[0]) to show the first data point, for instance.

In [ ]:
def download(filename, source = 'http://yann.lecun.com/exdb/mnist/index.html'):
    print("Downloading %s" % filename)
    urlretrieve(source + filename, filename)
    
def load_mnist_images(filename):
    if not os.path.exists(filename):
        download(filename)
    # Read the inputs in Yann LeCun's binary format.
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset = 16)
    data = data.reshape(-1,784)
    return data / np.float32(256)

def load_mnist_labels(filename):
    if not os.path.exists(filename):
        download(filename)
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset = 8)
    return data

def displaychar(image):
    plt.imshow(np.reshape(image, (28,28)), cmap=plt.cm.gray)
    plt.axis('off')
    plt.show()

In [ ]:
# Load the training data set
train_data = load_mnist_images('train-images-idx3-ubyte.gz')
train_labels = load_mnist_labels('train-labels-idx1-ubyte.gz')

print('Shape of the train data:\n', train_data.shape)
print('\nShape of the train labels:\n', train_labels.shape)

In [ ]:
# Display the first data point
displaychar(train_data[0])

#### (b) Split the training set into two pieces – a training set of size 50000, and a separate validation set of size 10000. Also load in the test data.

In [ ]:
x, y = train_data, train_labels

# randomly select 50000 data from data set as training data set.
random.seed(10)
sel = random.sample(range (0, x.shape[0]), 50000 )
x_train_data, y_train_data = train_data[sel,], train_labels[sel,]

print('Shape of x_train_data:\n', x_train_data.shape)
print('\nShape of y_train_data:\n', y_train_data.shape)

In [ ]:
# get the remaining data as validation data set.
remain = np.setdiff1d(range(0, x.shape[0]), sel)
x_validation, y_validation = train_data[remain,], train_labels[remain,]

print('Shape of x_validation:\n', x_validation.shape)
print('\nShape of y_validation:\n', y_validation.shape)

In [ ]:
# Load the testing data set
test_data = load_mnist_images('t10k-images-idx3-ubyte.gz')
test_labels = load_mnist_labels('t10k-labels-idx1-ubyte.gz')

print('Shape of the test data:\n', test_data.shape)
print('\nShape of the test labels:\n', test_labels.shape)

#### (c) Now fit a Gaussian generative model to the training data of 50000 points.

• Determine the class probabilities.

• Fit a Gaussian to each digit, by finding the mean and the covariance of the corresponding data points.

In [ ]:
# create a multivariabte Gaussian model

def MultivariateGaussian(x, y):
    #labels 1,2,...,k
    k = 10  
    
    #number of features
    d = (x.shape)[1]  
    
    mu = np.zeros((k,d))
    sigma = np.zeros((k,d,d))
    pi = np.zeros(k)
    
    for label in range(10):
        indices = np.where(y == label)
        indices = indices[0]
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(len(indices))/float(len(y))
    return mu, sigma, pi

In [ ]:
# get the mu, sigma, and pi
mu, sigma, pi = MultivariateGaussian(x_train_data, y_train_data)

In [ ]:
print('Class probabilities:\n', pi)

In [ ]:
print('Mean of the corresponding data points:\n ', mu)

In [ ]:
print('Covariance of the corresponding data points:\n ', sigma)

#### (d) One last step is needed: it is important to smooth the covariance matrices, and the usual way to do this is to add in cI, where c is some constant and I is the identity matrix. What value of c is right? Use the validation set to help you choose. That is, choose the value of c for which the resulting classifier makes the fewest mistakes on the validation set. What value of c did you get?

In [ ]:
# classify funtion using Bayes' rule

def decision(x, pi, mu, sigma): 
    prob = np.zeros((10, x.shape[0]))
    
    for i in range (0,10):
        prob[i,:] = pi[i]* multivariate_normal.pdf(x, mean = mu[i,:], cov = sigma[i,:,:])                                             
        #if the sigma matrix cannot be inversed, we need to add a np.eye() to avoid singular matrix error.
        #since we will smooth in step (d), so I didn't use np.eye() here.
        #prob[i,:] = pi[i]* multivariate_normal.pdf(x, mean = mu[i,:], cov = sigma[i,:,:] + np.eye(sigma.shape[1],sigma.shape[2]))

    pred = np.argmax(prob, axis = 0)

    return pred, prob

In [ ]:
# set c as a constant and I as the identity matix
# randomly give c a number
c = 0.1
iden = np.zeros((10,784,784))

for i in range (0,10) :
    iden[i,:,:] = np.diag([1] * 784) 

# smooth the covariance matrix
decision(x_validation[:5,], pi, mu, sigma + c * iden)

In [ ]:
c_grid = np.linspace(0.01, 0.3, num=20 )
validation_error =[]

for c in c_grid:
    pred = decision(x_validation, pi, mu, sigma + c * iden)[0]
    validation_error.append(1 - np.mean(pred == y_validation))

In [ ]:
# smaller erorr is desired
plt.plot(c_grid,validation_error)

# find the optimal c value
c_optim = c_grid[np.argmin(validation_error)]

print('The optimal of c is:\n', np.min(validation_error))

#### (e) Turn in an iPython notebook that includes:

    • All your code.
    
    • Error rate on the MNIST test set.
    
    • Out of the misclassified test digits, pick five at random and display them. For each instance, list the posterior probabilities Pr(y|x) of each of the ten classes.

In [ ]:
# calculate the error rate on the MNIST test set.
accuracy = np.mean(decision(test_data, pi, mu, sigma + c_optim * iden)[0] == test_labels)
error_rate = (1 - accuracy)*100

print('Error rate on the MNIST test set:\n{}%'.format(error_rate))

In [ ]:
pred, prob = decision(test_data,pi, mu, sigma + c_optim * iden)

In [ ]:
mis = np.where(pred != test_labels)[0]
random.seed(14)

for i in random.sample(range(0,len(mis)),5):
    print('posterior probabilities are:\n', prob[:,mis[i]]/np.sum(prob[:,mis[i]]))
    #print(np.where(np.argmax(prob[:,mis[i]]/np.sum(prob[:,mis[i]]))))
    print('\nmissclassified digits', test_labels[mis[i]])
    print('---')

# Worksheet 10 Problem 2

For this problem, we’ll be using the animals with attributes data set. Go to 

    http://attributes.kyb.tuebingen.mpg.de
    
and, under “Downloads”, choose the “base package” (the very first file in the list). Unzip it and look over the various text files.

About the dataset: This is a small dataset that has information on about 50 animals. The animals are listed in classes.txt. For each animal, the information consists of values for 85 features: does the animal have a tail, is it slow, does it have tusks, etc. The details of the features are in the predicates.txt. The full data consists of a 50 x 85 matrix of real values, in predicate-matrix-continuous.txt. There is also a binarized version of this data, in predicate-matrix-binary.txt.

In [ ]:
def download(filename, source = 'http://attributes.kyb.tuebingen.mpg.de'):
    print("Downloading %s" % filename)
    urlretrieve(source + filename, filename)
    
def load_data(filename):
    if not os.path.exists(filename):
        download(filename)
    with bz2.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset = 16)
    return data / np.float32(256)

In [ ]:
base_package_data = load_data('AwA-base.tar.bz2')

#### (a) Load the real-valued array, and also the animal names, into Python. Run k-means on the data (from sklearn.cluster) and ask for k = 10 clusters. For each cluster, list the animals in it. Does the clustering make sense?

In [ ]:
# classes with animal names.
classes_with_animal_names = pd.read_fwf("Animals_with_Attributes/classes.txt", header = None)[1].values
print('Shape of the classes.txt:\n', classes_with_animal_names.shape, '\n')
print('Data of the classes.txt:\n', classes_with_animal_names)

In [ ]:
# predicates with feature names.
predicates_with_feature_names = pd.read_fwf("Animals_with_Attributes/predicates.txt", header = None)[1].values
print('Shape of the predicates.txt:\n', predicates_with_feature_names.shape, '\n')
print('Data of the predicates.txt:\n', predicates_with_feature_names)

In [ ]:
# matrix of real values.
predicate_matrix = pd.read_fwf("Animals_with_Attributes/predicate-matrix-continuous.txt", header = None).values
print('Shape of the predicate-matrix-continuous.txt:\n', predicate_matrix.shape, '\n')
print('Data of the predicate-matrix-continuous.txt:\n', predicate_matrix)

In [ ]:
# add column names (feature names) for the predicate matrix of the real value.
classes_features = pd.DataFrame(data = predicate_matrix, columns = predicates_with_feature_names)

# add row index for predicate matrix of the real value.
classes_features.index = classes_with_animal_names
print('Shape of classes_features:\n', classes_features.shape, '\n')
print('Data of classes_features:\n', classes_features)

In [ ]:
# run k-means on the data, with k = 10.
kmeans = KMeans(n_clusters = 10, init = 'k-means++',  n_init = 10)
kmeans.fit(predicate_matrix, classes_with_animal_names)

In [ ]:
# label all animals
print('labels for animals:\n',kmeans.labels_)

In [ ]:
# cluster animals to their labels

clustering = {i:[] for i in range(0,10)}

for i,j in enumerate(classes_with_animal_names):
    clustering[kmeans.labels_[i]].append(j)

for i in range(0,10):
    print('label =',i,'\n')
    print('animals:',clustering[i],'\n')

### Conclusion for (a):

#### Does the clustering make sense?
#### --- Yes, it makes some sense.
#### --- From the clustering above, we can see that pets in label 7 are grouped close together as well as other labels. 
#### --- However, it is not perfect. we need to figure out optimal k or find other algorithms to cluster these animals better.

#### (b) Now hierarchically cluster this data, using scipy.cluster.hierarchy.linkage. Choose Ward’s method, and plot the resulting tree using the dendrogram method, setting the orientation parameter to ‘right’ and labeling each leaf with the corresponding animal name. You will run into a problem: the plot is too cramped because the default figure size is so small. To make it larger, preface your code with the following:
           from pylab import rcParams
           rcParams[’figure.figsize’] = 5, 10
#### (or try a different size if this doesn’t seem quite right). Does the hierarchical clustering seem sensible to you?

In [ ]:
# use scipy.cluster.hierarchy.linkage to hierarchically cluster this data.
hierarchically_cluster_link = linkage(predicate_matrix, method = 'ward');

# use dendrogram method to plot the resulting tree.
dendrogram(hierarchically_cluster_link, orientation = "right", labels = classes_with_animal_names)

# resize the figure
rcParams['figure.figsize'] = [40,40]

### Conclusion for (b):

#### Does the hierarchical clustering seem sensible to you?
#### --- Yes, it seems sensible to me.
#### --- For choosing the linkage, Ward’s method is the sensible default. It groups based on reducing the sum of squared distances of each observation from the average observation in a cluster.

#### (c) Turn in an iPython notebook with a transcript of all this experimentation.

#### See above for all this experimentation